In [1]:
import numpy as np
import json
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
import pandas as pd


In [2]:
# Define the ranges for the variables in the optimization problem
variable_ranges = [
    [0, 1, 3, 7],  # Incubation Time (discrete)
    (0.0, 1.5, 0.1),  # Concentration of methyl alpha glucopyranoside (continuous, 0.1 increments)
    (5.0, 7.0, 0.2),  # pH (continuous, 0.2 increments)
    (0, 30, 2),  # 6 kDa dextran (increments of 2 percent)
    (0.0, 1.0, 0.1),  # Trehalose (continuous, 0.1 increments)
    (0.0, 1.0, 0.1),  # EGCG (continuous, 0.1 increments)
    (0.0, 3.0, 0.1),  # Polaxamer 188 (continuous, 0.1 increments)
    (0.0, 2.5, 0.1),  # BSA (continuous, 0.1 increments)
    (0, 300, 50)  # Arginine (50 mM increments)
]

column_names = [
    "Incubation Time (hrs)", 
    "Concentration of methyl alpha glucopyranoside (M)", 
    "pH", 
    "6 kda dextran (% w/v)", 
    "Trehalose (M)", 
    "EGCG (% w/v)", 
    "Polaxamer 188 (% w/v)", 
    "BSA (% w/v)", 
    "Arginine (mM)"
]

population_size = 8
F = 0.8  # Differential weight (mutation factor)
CR = 0.9  # Crossover probability
max_generations = 15  # Maximum number of generations or iterations



In [3]:
def generate_initial_population(variable_ranges, population_size):
    population = []
    for _ in range(population_size):
        individual = []
        for var in variable_ranges:
            if isinstance(var, tuple):
                # Continuous variable: (start, end, step)
                start, end, step = var
                # Slightly expand the range to include both start and end
                expanded_start = start - step / 2
                expanded_end = end + step / 2
                value = np.round(np.random.uniform(expanded_start, expanded_end) / step) * step
                # Clip the value to ensure it's within [start, end]
                value = np.clip(value, start, end)
                individual.append(value)
            else:
                # Discrete variable: select random value from the list
                value = np.random.choice(var)
                individual.append(value)
        population.append(individual)
    return np.array(population)

def evaluate_fitness(population, viability_results):
    return np.array(viability_results)

# Differential mutation 
def differential_mutation(population, F, variable_ranges):
    mutated_population = []
    for i in range(len(population)):
        indices = list(range(len(population)))
        indices.remove(i)
        a, b, c = population[np.random.choice(indices, 3, replace=False)]
        mutant = []
        for idx, var in enumerate(variable_ranges):
            if isinstance(var, tuple):  # Continuous variable
                mutated_value = a[idx] + F * (b[idx] - c[idx])
                # Clip to the bounds and round to the nearest step
                start, end, step = var
                mutated_value = np.clip(mutated_value, start, end)
                mutated_value = np.round(mutated_value / step) * step
            else:  # Discrete variable: no mutation, pick from original values
                mutated_value = np.random.choice([a[idx], b[idx], c[idx]])
            mutant.append(mutated_value)
        mutated_population.append(mutant)
    return np.array(mutated_population)

# Differential crossover 
def differential_crossover(population, mutated_population, CR):
    offspring_population = []
    for i in range(len(population)):
        offspring = []
        for j in range(len(population[i])):
            if np.random.rand() < CR:
                offspring.append(mutated_population[i][j])
            else:
                offspring.append(population[i][j])
        offspring_population.append(offspring)
    return np.array(offspring_population)

In [4]:
# Functions to save and load the population and fitness scores from files
def save_experiment(population, fitness_scores, generation):
    # Generate a filename with the current date and time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"experiment_data_{timestamp}.json"
    
    data = {
        'population': population.tolist(),
        'fitness_scores': fitness_scores.tolist(),
        'generation': generation
    }
    
    with open(filename, 'w') as file:
        json.dump(data, file)
    
    print(f"Experiment saved to {filename}")

def load_experiment(filename='experiment_data.json'):
    print(f"Loading experiment from {filename}")
    if os.path.exists(filename):
        with open(filename, 'r') as file:
            data = json.load(file)
        population = np.array(data['population'])
        fitness_scores = np.array(data['fitness_scores'])
        generation = data['generation']
        print(f"Experiment loaded from {filename}")
        return population, fitness_scores, generation
    else:
        print(f"No saved experiment found at {filename}. Starting new experiment.")
        return None, None, 0



In [ ]:
# User prompt to load from file or start new experiment
load_choice = input("Do you want to load the previous experiment? (yes/no): ").strip().lower()
print(f'Choice: {load_choice}')
if load_choice == 'yes':
    population, fitness_scores, generation = load_experiment(filename=input("Enter the filename"))
    print(f'population: {population}, fitness_scores: {fitness_scores}, generation: {generation}')
    if population is None:
        population = generate_initial_population(variable_ranges, population_size)
        fitness_scores = np.zeros(population_size)
        generation = 0
else:
    population = generate_initial_population(variable_ranges, population_size)
    fitness_scores = np.zeros(population_size)
    generation = 0
# Main loop
while True:
    print(f"\nGeneration {generation + 1}")

    # Display current population configurations with cleaner formatting
    print("\nCurrent population configurations:")
    for idx, individual in enumerate(population):
        formatted_individual = ["{:.1f}".format(x) if isinstance(x, float) else "{:.0f}".format(x) for x in individual]
        print(f"Configuration {idx + 1}: {formatted_individual}")
    display(pd.DataFrame(population, columns=column_names))

    # Prompt for new viability scores
    viability_results = []
    response = input('Were fitness scores loaded from json? yes/no')
    if response == 'no':
        for i in range(len(population)):
            score = float(input(f"Enter the viability score for configuration {i + 1}: "))
            viability_results.append(score)
        # Evaluate fitness
        fitness_scores = evaluate_fitness(population, viability_results)
    #else: fitness scores are already loaded so no evaluate_fitness necessary
    else:
        print(f'fitness scores were loaded: {fitness_scores}')

    # Perform differential evolution
    mutated_population = differential_mutation(population, F, variable_ranges)
    offspring_population = differential_crossover(population, mutated_population, CR)

    # Select the next generation by keeping the best performing individuals
    #combined_population = np.vstack((population, offspring_population))
    #combined_fitness_scores = np.concatenate((fitness_scores, evaluate_fitness(offspring_population, viability_results)))
    #sorted_indices = np.argsort(combined_fitness_scores)[:population_size]  # Minimization (select lowest)
    #population = combined_population[sorted_indices

    population = offspring_population

    generation += 1

    # Save the experiment after each generation with a unique filename
    save_experiment(population, fitness_scores, generation)

    # Option to continue or stop
    if generation >= max_generations:
        print("\nMaximum number of generations reached.")
        break

    continue_choice = input("Do you want to continue to the next generation? (yes/no): ").strip().lower()
    if continue_choice != 'yes':
        print("Stopping the process.")
        break

# Final result
print("\nFinal population configurations:")
for idx, individual in enumerate(population):
    formatted_individual = ["{:.1f}".format(x) if isinstance(x, float) else "{:.0f}".format(x) for x in individual]
    print(f"Configuration {idx + 1}: {formatted_individual}")


Do you want to load the previous experiment? (yes/no):  yes


Choice: yes


Enter the filename Gen3FromAli.json


Loading experiment from Gen3FromAli.json
Experiment loaded from Gen3FromAli.json
population: [[0.0e+00 1.5e+00 5.6e+00 6.0e+00 1.0e+00 0.0e+00 1.9e+00 2.5e+00 3.0e+02]
 [7.0e+00 1.3e+00 6.8e+00 6.0e+00 9.0e-01 2.0e-01 7.0e-01 1.6e+00 3.0e+02]
 [0.0e+00 6.0e-01 6.4e+00 3.0e+01 0.0e+00 1.0e+00 9.0e-01 7.0e-01 0.0e+00]
 [0.0e+00 1.1e+00 7.0e+00 2.0e+00 1.0e+00 0.0e+00 1.8e+00 1.3e+00 1.0e+02]
 [7.0e+00 3.0e-01 5.4e+00 1.8e+01 6.0e-01 6.0e-01 7.0e-01 8.0e-01 0.0e+00]
 [7.0e+00 4.0e-01 5.6e+00 1.8e+01 4.0e-01 0.0e+00 1.0e-01 1.7e+00 3.0e+02]
 [7.0e+00 3.0e-01 6.6e+00 4.0e+00 9.0e-01 0.0e+00 9.0e-01 9.0e-01 2.0e+02]
 [0.0e+00 1.0e+00 5.0e+00 2.8e+01 2.0e-01 1.0e+00 7.0e-01 6.0e-01 2.0e+02]], fitness_scores: [1.02 0.88 0.   0.96 0.22 0.79 0.98 0.  ], generation: 3

Generation 4

Current population configurations:
Configuration 1: ['0.0', '1.5', '5.6', '6.0', '1.0', '0.0', '1.9', '2.5', '300.0']
Configuration 2: ['7.0', '1.3', '6.8', '6.0', '0.9', '0.2', '0.7', '1.6', '300.0']
Configuration 3:

,Incubation Time (hrs),Concentration of methyl alpha glucopyranoside (M),pH,6 kda dextran (% w/v),Trehalose (M),EGCG (% w/v),Polaxamer 188 (% w/v),BSA (% w/v),Arginine (mM)
0,0.0,1.5,5.6,6.0,1.0,0.0,1.9,2.5,300.0
1,7.0,1.3,6.8,6.0,0.9,0.2,0.7,1.6,300.0
2,0.0,0.6,6.4,30.0,0.0,1.0,0.9,0.7,0.0
3,0.0,1.1,7.0,2.0,1.0,0.0,1.8,1.3,100.0
4,7.0,0.3,5.4,18.0,0.6,0.6,0.7,0.8,0.0
5,7.0,0.4,5.6,18.0,0.4,0.0,0.1,1.7,300.0
6,7.0,0.3,6.6,4.0,0.9,0.0,0.9,0.9,200.0
7,0.0,1.0,5.0,28.0,0.2,1.0,0.7,0.6,200.0


...


Generation 5

Current population configurations:
Configuration 1: ['0.0', '1.5', '6.6', '20.0', '0.4', '0.6', '0.9', '0.9', '0.0']
Configuration 2: ['0.0', '0.8', '7.0', '4.0', '0.8', '0.0', '2.0', '1.4', '0.0']
Configuration 3: ['7.0', '1.0', '7.0', '0.0', '1.0', '0.2', '1.4', '0.8', '200.0']
Configuration 4: ['7.0', '1.5', '5.0', '18.0', '0.7', '1.0', '1.8', '1.2', '100.0']
Configuration 5: ['7.0', '0.7', '5.4', '4.0', '1.0', '0.0', '2.1', '1.9', '200.0']
Configuration 6: ['7.0', '1.5', '6.2', '18.0', '0.4', '0.7', '0.7', '1.2', '300.0']
Configuration 7: ['0.0', '0.8', '5.0', '18.0', '0.7', '0.7', '0.5', '0.7', '200.0']
Configuration 8: ['0.0', '0.2', '5.8', '0.0', '1.0', '0.0', '0.1', '1.9', '300.0']


,Incubation Time (hrs),Concentration of methyl alpha glucopyranoside (M),pH,6 kda dextran (% w/v),Trehalose (M),EGCG (% w/v),Polaxamer 188 (% w/v),BSA (% w/v),Arginine (mM)
0,0.0,1.5,6.6,20.0,0.4,0.6,0.9,0.9,0.0
1,0.0,0.8,7.0,4.0,0.8,0.0,2.0,1.4,0.0
2,7.0,1.0,7.0,0.0,1.0,0.2,1.4,0.8,200.0
3,7.0,1.5,5.0,18.0,0.7,1.0,1.8,1.2,100.0
4,7.0,0.7,5.4,4.0,1.0,0.0,2.1,1.9,200.0
5,7.0,1.5,6.2,18.0,0.4,0.7,0.7,1.2,300.0
6,0.0,0.8,5.0,18.0,0.7,0.7,0.5,0.7,200.0
7,0.0,0.2,5.8,0.0,1.0,0.0,0.1,1.9,300.0


Do you want to continue to the next generation? (yes/no):  yes



Generation 3

Current population configurations:
Configuration 1: ['0.0', '1.5', '5.6', '6.0', '1.0', '0.0', '1.9', '2.5', '300.0']
Configuration 2: ['7.0', '1.3', '6.8', '6.0', '0.9', '0.2', '0.7', '1.6', '300.0']
Configuration 3: ['0.0', '0.6', '6.4', '30.0', '0.0', '1.0', '0.9', '0.7', '0.0']
Configuration 4: ['0.0', '1.1', '7.0', '2.0', '1.0', '0.0', '1.8', '1.3', '100.0']
Configuration 5: ['7.0', '0.3', '5.4', '18.0', '0.6', '0.6', '0.7', '0.8', '0.0']
Configuration 6: ['7.0', '0.4', '5.6', '18.0', '0.4', '0.0', '0.1', '1.7', '300.0']
Configuration 7: ['7.0', '0.3', '6.6', '4.0', '0.9', '0.0', '0.9', '0.9', '200.0']
Configuration 8: ['0.0', '1.0', '5.0', '28.0', '0.2', '1.0', '0.7', '0.6', '200.0']


,Incubation Time (hrs),Concentration of methyl alpha glucopyranoside (M),pH,6 kda dextran (% w/v),Trehalose (M),EGCG (% w/v),Polaxamer 188 (% w/v),BSA (% w/v),Arginine (mM)
0,0.0,1.5,5.6,6.0,1.0,0.0,1.9,2.5,300.0
1,7.0,1.3,6.8,6.0,0.9,0.2,0.7,1.6,300.0
2,0.0,0.6,6.4,30.0,0.0,1.0,0.9,0.7,0.0
3,0.0,1.1,7.0,2.0,1.0,0.0,1.8,1.3,100.0
4,7.0,0.3,5.4,18.0,0.6,0.6,0.7,0.8,0.0
5,7.0,0.4,5.6,18.0,0.4,0.0,0.1,1.7,300.0
6,7.0,0.3,6.6,4.0,0.9,0.0,0.9,0.9,200.0
7,0.0,1.0,5.0,28.0,0.2,1.0,0.7,0.6,200.0
